In [1]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import requests
from pytrends.request import TrendReq

# Fetching Time Series Data

In [6]:
stock = "AXISBANK.NS"
searchterm = "Axis Bank"
begin_date = "2015-07-01"
end_date = "2018-04-30"
filename="AxisBankwTS.csv"

API_KEY = "86GWLA2M8WL2J702"
ts = TimeSeries(key=API_KEY)

url = "https://www.alphavantage.co/query"
print("Fetching Time Series Data")

symbol = stock
function = "TIME_SERIES_DAILY_ADJUSTED"
api_key = "86GWLA2M8WL2J702"

data = { "function": function, 
    "symbol": symbol,
     "outputsize": "full",
     "datatype": "csv",
     "apikey": api_key}

page = requests.get(url, params = data)

with open(filename, 'w') as oF:
    oF.write(page.text)
    
Data = pd.read_csv(filename, index_col=None)

#Rename the Columns
Data.rename(columns={'timestamp':'Date','open':'Open','high':'High','low':'Low','close':'Close','adjusted_close':'Adj_Close','volume':'Volume','dividend_amount':'Dividend_Amt','split_coefficient':'Split_Coeff'}, inplace=True)

#Drop The Columns 'Dividend_Amt', 'Split_Coeff'
Data = Data.drop(['Dividend_Amt', 'Split_Coeff'], axis =1)

#Drop the rows with value 0 for open
Data = Data[Data["Open"]!=0]

#Reverse th Data i.e. from 2000 to 2018
Data = Data.iloc[::-1]

#Reset The Index and Save It
Data = Data.reset_index(drop=True)
Data.to_csv("AxisBankwTS.csv")

Fetching Time Series Data


In [5]:
Data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2000-01-03,5.10,5.18,5.10,5.18,0.9778,82500
1,2000-01-04,5.40,5.51,5.08,5.38,1.0156,494000
2,2000-01-05,5.01,5.42,5.00,5.24,0.9892,383000
3,2000-01-06,5.30,5.30,5.00,5.11,0.9646,164500
4,2000-01-07,5.00,5.22,4.80,4.86,0.9174,149000


# Fetching Technical Indicators Data

In [7]:
print("Fetchin Stochastic")
#Fetch Stoch
data_stoch = { "function": "STOCH", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "apikey": api_key}
page = requests.get(url, params = data_stoch)
temp = page.json()
tempdata = temp['Technical Analysis: STOCH']

tempDate = tempdata.keys()
tempValues = tempdata.values()
SlowD=[]
SlowK=[]
for ele in tempValues:
    SlowD.append(ele['SlowD'])
    SlowK.append(ele['SlowK'])
    
Stoch_df = pd.DataFrame()
Stoch_df['Date']=tempDate
Stoch_df['SlowD']=SlowD
Stoch_df['SlowK']=SlowK

Fetchin Stochastic


In [8]:
Data['Date']=pd.to_datetime(Data.Date)
Stoch_df['Date']=pd.to_datetime(Stoch_df.Date)
Data = Data.merge(Stoch_df, how ='inner', left_on = 'Date', right_on = 'Date') 

In [6]:
print("Fetching WILLR")
data_willr = { "function": "WILLR", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "time_period":"10",
     "apikey": api_key}
page = requests.get(url, params = data_willr)
temp_willr = page.json()
tempdata_willr = temp_willr['Technical Analysis: WILLR']

tempDate_willr = tempdata_willr.keys()
tempValues_willr = tempdata_willr.values()
Willr=[]
for ele in tempValues_willr:
    Willr.append(ele)
    
Willr_var=[]
for j in Willr:
    Willr_var.append(j['WILLR'])
Willr_df = pd.DataFrame()
Willr_df['Date']=tempDate_willr
Willr_df['WILLR']=Willr_var

Fetching WILLR


In [7]:
Data['Date']=pd.to_datetime(Data.Date)
Willr_df['Date']=pd.to_datetime(Willr_df.Date)
Data = Data.merge(Willr_df, how ='inner', on = 'Date')

In [8]:
print("Fetching MACD")
#Fetch MACD
data = { "function": "MACD", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "series_type": "open",
     "apikey": api_key}
page = requests.get(url, params = data)
temp = page.json()
tempdata = temp['Technical Analysis: MACD']

tempDate = tempdata.keys()
tempValues = tempdata.values()
MACD_Signal=[]
MACD_Hist=[]
MACD=[]
for ele in tempValues:
    MACD.append(ele['MACD'])
    MACD_Signal.append(ele['MACD_Signal'])
    MACD_Hist.append(ele['MACD_Hist'])
    
MACD_df = pd.DataFrame()
MACD_df['Date']=tempDate
MACD_df['MACD']=MACD
MACD_df['MACD_Signal']=MACD_Signal
MACD_df['MACD_Hist']=MACD_Hist

Fetching MACD


In [9]:
Data['Date']=pd.to_datetime(Data.Date)
MACD_df['Date']=pd.to_datetime(MACD_df.Date)
Data = Data.merge(MACD_df, how ='inner', on = 'Date') 

In [10]:
print("Fetching RSI")
#Get RSI
data = { "function": "RSI", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "series_type": "open",
     "time_period":"10",
     "apikey": api_key}
page = requests.get(url, params = data)
temp = page.json()
tempdata = temp['Technical Analysis: RSI']

tempDate = tempdata.keys()
tempValues = tempdata.values()
RSI=[]
for ele in tempValues:
    RSI.append(ele['RSI'])

    
RSI_df = pd.DataFrame()
RSI_df['Date']=tempDate
RSI_df['RSI']=RSI

Fetching RSI


In [11]:
Data['Date']=pd.to_datetime(Data.Date)
RSI_df['Date']=pd.to_datetime(RSI_df.Date)
Data = Data.merge(RSI_df, how ='inner', on = 'Date') 

In [12]:
print("Fetching ADX")
data = { "function": "ADX", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "time_period":"10",
     "apikey": api_key}
page = requests.get(url, params = data)
temp = page.json()
tempdata = temp['Technical Analysis: ADX']

tempDate = tempdata.keys()
tempValues = tempdata.values()
ADX=[]
for ele in tempValues:
    ADX.append(ele['ADX'])

    
ADX_df = pd.DataFrame()
ADX_df['Date']=tempDate
ADX_df['ADX']=ADX

Fetching ADX


In [13]:
Data['Date']=pd.to_datetime(Data.Date)
ADX_df['Date']=pd.to_datetime(ADX_df.Date)
Data = Data.merge(ADX_df, how ='inner', on = 'Date')

In [14]:
print("Fetching CCI")
#Get CCI
data = { "function": "CCI", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "time_period":"10",
     "apikey": api_key}
page = requests.get(url, params = data)
temp = page.json()
tempdata = temp['Technical Analysis: CCI']

tempDate = tempdata.keys()
tempValues = tempdata.values()
CCI=[]
for ele in tempValues:
    CCI.append(ele['CCI'])

    
CCI_df = pd.DataFrame()
CCI_df['Date']=tempDate
CCI_df['CCI']=CCI

Fetching CCI


In [15]:
Data['Date']=pd.to_datetime(Data.Date)
CCI_df['Date']=pd.to_datetime(CCI_df.Date)
Data = Data.merge(CCI_df, how ='inner', on = 'Date') 

In [16]:
print("Fetching AD")
data = { "function": "AD", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "apikey": api_key}
page = requests.get(url, params = data)
temp = page.json()
tempdata = temp['Technical Analysis: Chaikin A/D']

tempDate = tempdata.keys()
tempValues = tempdata.values()
AD=[]
for ele in tempValues:
    AD.append(ele['Chaikin A/D'])

    
AD_df = pd.DataFrame()
AD_df['Date']=tempDate
AD_df['Chaikin A/D']=AD

Fetching AD


In [17]:
Data['Date']=pd.to_datetime(Data.Date)
AD_df['Date']=pd.to_datetime(AD_df.Date)
Data = Data.merge(AD_df, how ='inner', on = 'Date') 

In [19]:
print("Fetching OBV")
#Fetch OBV
data = { "function": "OBV", 
    "symbol": symbol,
     "outputsize": "full",
     "interval": "daily",
     "apikey": api_key}
page = requests.get(url, params = data)
temp = page.json()
tempdata = temp['Technical Analysis: OBV']

tempDate = tempdata.keys()
tempValues = tempdata.values()
OBV=[]
for ele in tempValues:
    OBV.append(ele['OBV'])
    
OBV_df = pd.DataFrame()
OBV_df['Date']=tempDate
OBV_df['OBV']=OBV

Fetching OBV


In [20]:
Data['Date']=pd.to_datetime(Data.Date)
OBV_df['Date']=pd.to_datetime(OBV_df.Date)
Data = Data.merge(OBV_df, how ='inner', on = 'Date') 

In [21]:
Data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume,SlowD,SlowK,WILLR,MACD,MACD_Signal,MACD_Hist,RSI,ADX,CCI,Chaikin A/D,OBV
0,2000-02-17,7.80,7.86,7.46,7.50,1.4158,282000,24.1228,19.7531,-78.5000,0.7759,0.7358,0.0401,55.0398,23.2628,-50.8876,-4788646.2455,-1112500.0000
1,2000-02-18,7.40,7.55,6.91,7.05,1.3308,448500,20.4926,20.2658,-93.5185,0.7222,0.7331,-0.0108,51.9009,21.4223,-115.7220,-5040927.4955,-1561000.0000
2,2000-02-21,7.49,7.60,7.10,7.19,1.3573,360500,19.9012,19.6848,-86.3415,0.6792,0.7223,-0.0431,52.5770,19.8626,-77.4985,-5271647.4955,-1200500.0000
3,2000-02-22,7.44,7.50,6.70,6.92,1.3063,331500,19.4578,18.4228,-90.2655,0.6337,0.7046,-0.0709,52.1247,18.1711,-111.5418,-5420822.4955,-1532000.0000
4,2000-02-23,6.78,7.14,6.60,6.74,1.2723,227500,19.3192,19.8501,-94.0678,0.5382,0.6713,-0.1331,46.2850,16.8671,-119.6643,-5530359.5325,-1759500.0000


In [22]:
Data.to_csv('AxisBankwTech.csv')

# Fetching Google Trends Data

In [30]:
def SMA(values, window):
    weights = np.repeat(1.0, window)/window
    smas = np.convolve(values,weights,'valid')
    return smas

def EMA(values, window):
    weights = np.exp(np.linspace(-1.,0.,window))
    weights /= weights.sum()
    
    a = np.convolve(values, weights)[:len(values)]
    a[:window]=[window]
    return a

In [29]:
print("Fetching Google Trends Data")
#Get Google Trends Data
pytrends = TrendReq(hl='en-US', tz=360)
#X = Reliance_Data['Date'].to_frame()
#X['Date'] = pd.to_datetime(X.Date)
#X['Date'] = X['Date'].dt.date
#print(X['Date'])
#X['Date']=sorted(X['Date'], reverse = False)
#Data = Data.iloc[::-1]
#Data = Data.reset_index()
X = pd.DataFrame()
X = Data
X = X.iloc[::-1]
X = X['Date'].iloc[0:3000].to_frame()
X['Date'] = pd.to_datetime(X.Date)
X['Date'] = X['Date'].dt.date
X['Date']=sorted(X['Date'], reverse = False)

Google_Data = pd.DataFrame()
x = pd.DataFrame()
final = []
kw_list = [searchterm]
i=0
while ( i < len(X) ):
    if(i>len(X)-90):
        print(str(X['Date'].iloc[i])+' '+str(X['Date'].iloc[-1]))
        pytrends.build_payload(kw_list, cat=7, timeframe=str(X['Date'].iloc[i])+' '+str(X['Date'].iloc[-1]), geo='IN', gprop='')
        i=i+91
        x=pytrends.interest_over_time()
        x['Date']=x.index
        Google_Data=Google_Data.append(x, ignore_index= True)
    else:
        print(str(X['Date'].iloc[i])+' '+str(X['Date'].iloc[i+90]))
        pytrends.build_payload(kw_list, cat=7, timeframe=str(X['Date'].iloc[i])+' '+str(X['Date'].iloc[i+90]), geo='IN', gprop='')
        i=i+91
        x=pytrends.interest_over_time()
        x['Date']=x.index
        Google_Data=Google_Data.append(x, ignore_index= True)

Fetching Google Trends Data
2006-06-14 2006-10-20
2006-10-23 2007-03-07
2007-03-08 2007-07-18


C:\Users\Avi\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


2007-07-19 2007-11-27
2007-11-28 2008-04-09
2008-04-10 2008-08-21
2008-08-22 2009-01-12
2009-01-13 2009-06-01
2009-06-02 2009-10-09
2009-10-12 2010-02-24
2010-02-25 2010-07-07
2010-07-08 2010-11-15
2010-11-16 2011-03-28
2011-03-29 2011-08-05
2011-08-08 2011-12-23
2011-12-26 2012-05-08
2012-05-09 2012-09-14
2012-09-17 2013-01-31
2013-02-01 2013-06-14
2013-06-17 2013-10-28
2013-10-29 2014-03-10
2014-03-11 2014-07-22
2014-07-23 2014-12-10
2014-12-11 2015-04-27
2015-04-28 2015-09-02
2015-09-03 2016-01-19
2016-01-20 2016-06-03
2016-06-06 2016-10-18
2016-10-19 2017-02-28
2017-03-01 2017-07-12
2017-07-13 2017-11-22
2017-11-23 2018-04-06
2018-04-09 2018-08-10


NameError: name 'SMA' is not defined

In [31]:
templist = [0,0]
templist = templist + list(SMA(Google_Data[searchterm],3))
Google_Data['SMA3_GoogleTrends'] = pd.Series(templist)
Google_Data['EMA3_GoogleTrends'] = EMA(Google_Data[searchterm],3)
Google_Data['SMA5_GoogleTrends'] = pd.Series([0,0,0,0]+list(SMA(Google_Data[searchterm],5)))
Google_Data['EMA5_GoogleTrends'] = EMA(Google_Data[searchterm],5)

In [32]:
Google_Data.to_csv('GoogleTrendsIndia.csv')

In [38]:
Data['Date']=pd.to_datetime(Data.Date)
Google_Data['Date']=pd.to_datetime(Google_Data.Date)
Data = Data.merge(Google_Data, how ='inner', on = 'Date')

In [40]:
Data

,Date,Open,High,Low,Close,Adj_Close,Volume,SlowD,SlowK,WILLR,...,ADX,CCI,Chaikin A/D,OBV,Axis Bank,isPartial,SMA3_GoogleTrends,EMA3_GoogleTrends,SMA5_GoogleTrends,EMA5_GoogleTrends
0,2007-03-08,84.73,90.00,82.41,87.22,53.0452,4318735,31.1875,28.0967,-69.6843,...,43.0987,-85.9803,-88695641.9275,387207475.0000,0.0,False,0.000000,3.000000,0.0,5.000000
1,2007-03-09,87.98,93.58,85.40,91.28,55.5144,4839985,34.1700,48.6871,-44.2022,...,43.6819,-26.3314,-86577412.4336,392047460.0000,0.0,False,0.000000,3.000000,0.0,5.000000
2,2007-03-12,90.20,94.59,90.20,91.81,55.8368,3205705,48.7047,69.3304,-40.6147,...,43.6938,23.4586,-87431778.2849,395253165.0000,0.0,False,0.000000,0.000000,0.0,5.000000
3,2007-03-13,91.42,100.24,91.42,98.72,60.0393,5450040,67.9299,85.7721,-7.2415,...,41.1686,125.3260,-83860206.4141,400703205.0000,0.0,False,0.000000,0.000000,0.0,0.000000
4,2007-03-14,97.00,97.00,92.70,94.56,57.5093,6524695,78.6763,80.9265,-27.0605,...,38.8958,65.6804,-84740282.4570,394178510.0000,0.0,False,0.000000,0.000000,0.0,0.000000
5,2007-03-15,96.39,99.28,93.01,93.86,57.0835,2745740,79.7784,72.6367,-30.3954,...,35.8938,78.4550,-86741563.9655,391432770.0000,0.0,False,0.000000,0.000000,0.0,0.000000
6,2007-03-16,95.14,95.40,87.30,89.74,54.5779,3427260,67.1886,48.0026,-50.0238,...,34.8641,-1.4670,-88104009.5714,388005510.0000,0.0,False,0.000000,0.000000,0.0,0.000000
7,2007-03-19,91.00,95.10,88.42,94.01,57.1748,2603500,54.4041,42.5730,-29.6808,...,33.9374,28.2352,-86350151.6630,390609010.0000,0.0,False,0.000000,0.000000,0.0,0.000000
8,2007-03-20,96.00,96.20,91.70,92.56,56.2929,2271440,42.9272,38.2058,-36.5889,...,32.5781,39.7654,-87753395.8090,388337570.0000,0.0,False,0.000000,0.000000,0.0,0.000000
9,2007-03-21,93.00,97.00,92.60,96.22,58.5188,3382785,45.8395,56.7395,-22.5463,...,30.9614,70.4999,-85569959.7717,391720355.0000,0.0,False,0.000000,0.000000,0.0,0.000000


In [41]:
Data.to_csv('AxisBankwGoogle.csv')